## Rating insights

In [1]:
import pandas as pd
import logging
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Legend

output_notebook()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


logging.basicConfig(
    level=logging.INFO,
    format = '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s - %(lineno)d - %(message)s',
    handlers=[logging.StreamHandler()]
)

file = input('Enter file name: ')
ratings_df = pd.read_csv(file)
ratings_df['Ordered time'] = ratings_df['Ordered time'].apply(lambda x: x.split(' +')[0])
min_date = pd.to_datetime(min(ratings_df['Ordered time'])).strftime('%Y-%m-%d')
max_date = pd.to_datetime(max(ratings_df['Ordered time'])).strftime('%Y-%m-%d')
logging.info('Minimum date in the data: {} | Maximum date in the data: {}'.format(min_date, max_date))
logging.info('Select start and end dates within the min and max dates')
#start_date = input('Enter start time: ')
#start_date = start_date + ' 00:00:00'
#end_date = input('Enter end date: ')
#end_date = end_date + ' 23:59:59'
start_time = input('Enter start time (2019-01-30 00:00:00): ')
end_time = input('Enter end time (2019-01-30 11:59:59): ')

logging.info('==== processing raw file ====')
ratings_df = ratings_df[(start_time <= ratings_df['Ordered time']) & (ratings_df['Ordered time'] <= end_time)]
ratings_df = ratings_df.dropna(subset=['Items'])
logging.info('No of orders in {}: {}'.format(file, len(ratings_df['Order number'])))

cols = ratings_df.columns
ratings_df['Entity List'] = ratings_df['Items'].apply(lambda x: x.split('; '))
entity_cols = ratings_df.columns

entity_df = pd.DataFrame(ratings_df['Entity List'].values.tolist(), index=ratings_df.index).merge(ratings_df, left_index=True, right_index=True).melt(id_vars=entity_cols, value_name='Entity').drop('variable', axis=1).dropna(subset=['Entity'])
logging.info('No of orders in entity_df: {}'.format(len(entity_df['Order number'].unique())))

entity_df['Issues'] = entity_df['Issues'].fillna('')
entity_df['Issues List'] = entity_df['Issues'].apply(lambda x: x.split(', '))
issue_cols = entity_df.columns

issue_df = pd.DataFrame(entity_df['Issues List'].values.tolist(), index=entity_df.index).merge(entity_df, left_index=True, right_index=True).melt(id_vars=issue_cols, value_name='Issue').drop('variable', axis=1).dropna(subset=['Issue'])
logging.info('No of orders in issue_df: {}'.format(len(issue_df['Order number'].unique())))

issue_df = issue_df.merge(pd.DataFrame(issue_df['Entity'].apply(lambda x: x.replace(' - ', ' x ').split(' x ')).values.tolist(), index=issue_df.index, columns=['Item', 'Quantity', 'Cart Price']), left_index=True, right_index=True)

logging.info(' ==== finished processing raw file ==== ')

rests = ['Shaikpet', 'Kondapur', 'Yousufguda', 'Gachibowli', 'Madhapur', 'Istahgrill Himayat Nagar']
logging.info('Fetching ratings data for: {}'.format(rests))

rests_issue_df = issue_df[issue_df['Outlet name'].isin(rests)].copy()

rests_issue_df['Ordered time'] = rests_issue_df['Ordered time'].apply(lambda x: x.replace(' +0530', ''))
rests_issue_df['Ordered time'] = pd.to_datetime(rests_issue_df['Ordered time'])
rests_issue_df['Shift'] = rests_issue_df['Ordered time'].apply(lambda x: 'M' if 8 <= x.hour <= 18 else 'E')

min_time = min(rests_issue_df['Ordered time'])
max_time = max(rests_issue_df['Ordered time'])

logging.info('No of orders in rests_issue_df: {}'.format(len(rests_issue_df['Order number'].unique())))
logging.info('Orders start date: {} | Orders end date: {}'.format(min_time, max_time))


Loading BokehJS ...

Enter file name: online-ratings-2019-04-25-31.csv


2019-04-03 17:52:36,372 - root - INFO - <module> - 24 - Minimum date in the data: 2018-11-30 | Maximum date in the data: 2019-03-31
2019-04-03 17:52:36,373 - root - INFO - <module> - 25 - Select start and end dates within the min and max dates


Enter start time (2019-01-30 00:00:00): 2019-03-25 00:00:00
Enter end time (2019-01-30 11:59:59): 2019-04-01 00:00:00


2019-04-03 17:52:52,187 - root - INFO - <module> - 33 - ==== processing raw file ====
2019-04-03 17:52:52,199 - root - INFO - <module> - 36 - No of orders in online-ratings-2019-04-25-31.csv: 2102
2019-04-03 17:52:52,285 - root - INFO - <module> - 43 - No of orders in entity_df: 2102
2019-04-03 17:52:52,451 - root - INFO - <module> - 50 - No of orders in issue_df: 2102
2019-04-03 17:52:52,474 - root - INFO - <module> - 54 -  ==== finished processing raw file ==== 
2019-04-03 17:52:52,475 - root - INFO - <module> - 57 - Fetching ratings data for: ['Shaikpet', 'Kondapur', 'Yousufguda', 'Gachibowli', 'Madhapur', 'Istahgrill Himayat Nagar']
2019-04-03 17:52:52,601 - root - INFO - <module> - 68 - No of orders in rests_issue_df: 2085
2019-04-03 17:52:52,602 - root - INFO - <module> - 69 - Orders start date: 2019-03-25 00:03:09 | Orders end date: 2019-03-31 23:34:16


# By Shift

In [2]:
print("RATINGS SPLIT BY OUTLET")
ratings_shift_count = rests_issue_df.groupby(['Outlet name', 'Shift', 'Rating'])['Order number'].nunique().unstack(level=[0,1])
ratings_shift_count.loc['Total', :] = ratings_shift_count.sum()
ratings_shift_count.fillna(0)


RATINGS SPLIT BY OUTLET


Outlet name Gachibowli        Istahgrill Himayat Nagar        Kondapur        Madhapur        Shaikpet       Yousufguda      
Shift                E      M                        E      M        E      M        E      M        E     M          E     M
Rating                                                                                                                       
1.0               10.0   12.0                      9.0   14.0      8.0    8.0     19.0   11.0     10.0   2.0        9.0   7.0
2.0               10.0   10.0                      3.0    5.0      6.0    1.0     10.0   11.0      7.0   3.0        7.0   3.0
3.0               24.0   12.0                     17.0   15.0     30.0   16.0     23.0   12.0     18.0   9.0       19.0  16.0
4.0               59.0   31.0                     51.0   33.0     46.0   16.0     59.0   29.0     23.0  19.0       35.0  17.0
5.0              154.0   90.0                    110.0   90.0    141.0   74.0    215.0   99.0     81.0  29.0      128.0  50.0
Total            257.0  155.0                    190.0  157.0    231.0  115.0    326.0  162.0    139.0  62.0      198.0  93.0

In [3]:
print("PERCENTAGE OF RATINGS SPLIT BY OUTLET")
ratings_shift_count.div(ratings_shift_count.loc['Total', :], axis=1).drop('Total', axis='index').round(2)*100

PERCENTAGE OF RATINGS SPLIT BY OUTLET


Outlet name Gachibowli       Istahgrill Himayat Nagar       Kondapur       Madhapur       Shaikpet       Yousufguda      
Shift                E     M                        E     M        E     M        E     M        E     M          E     M
Rating                                                                                                                   
1.0                4.0   8.0                      5.0   9.0      3.0   7.0      6.0   7.0      7.0   3.0        5.0   8.0
2.0                4.0   6.0                      2.0   3.0      3.0   1.0      3.0   7.0      5.0   5.0        4.0   3.0
3.0                9.0   8.0                      9.0  10.0     13.0  14.0      7.0   7.0     13.0  15.0       10.0  17.0
4.0               23.0  20.0                     27.0  21.0     20.0  14.0     18.0  18.0     17.0  31.0       18.0  18.0
5.0               60.0  58.0                     58.0  57.0     61.0  64.0     66.0  61.0     58.0  47.0       65.0  54.0

In [4]:
print("AVERAGE RATING PER OUTLET (BLUE = < 4.3, RED = < 4)")
shift_orders = rests_issue_df.groupby(['Outlet name', 'Shift', 'Order number'])['Rating'].mean().reset_index()


avg_shift_rating = shift_orders.groupby(['Outlet name', 'Shift'])['Rating'].agg(['mean', 'count']).unstack(level=-1)
avg_shift_rating.loc['Avg Rating', pd.IndexSlice['mean', 'E']] = shift_orders['Rating'][shift_orders['Shift'] == 'E'].mean()
avg_shift_rating.loc['Avg Rating', pd.IndexSlice['mean', 'M']] = shift_orders['Rating'][shift_orders['Shift'] == 'M'].mean()
avg_shift_rating.fillna(0).round(2).style.apply(lambda x: ['color: red' if 0 < v < 4 else ('color: blue' if 4 <= v < 4.3 else '') for v in x], axis=1)



AVERAGE RATING PER OUTLET (BLUE = < 4.3, RED = < 4)


In [5]:
print('NUMBER OF BAD ISSUES OVERALL')
issues=['missing_item', 'negative_food_packaging', 'negative_portion_size', 'negative_taste', 'negative_wrong_or_missing_item']
issues_shift_count = rests_issue_df.groupby(['Issue', 'Outlet name', 'Shift'])['Order number'].nunique().round(2).unstack(level=[1,2])
issues_shift_count = issues_shift_count.loc[issues,:].fillna(0)
issues_shift_count



NUMBER OF BAD ISSUES OVERALL


Outlet name                    Gachibowli       Istahgrill Himayat Nagar       Kondapur      Madhapur       Shaikpet      Yousufguda     
Shift                                   E     M                        E     M        E    M        E     M        E    M          E    M
Issue                                                                                                                                    
missing_item                          0.0   0.0                      1.0   0.0      0.0  0.0      0.0   1.0      0.0  0.0        1.0  0.0
negative_food_packaging               1.0   3.0                      1.0   3.0      4.0  5.0      4.0   2.0      2.0  0.0        2.0  5.0
negative_portion_size                 4.0   8.0                      3.0   8.0      7.0  2.0      4.0   4.0      7.0  0.0        3.0  4.0
negative_taste                       13.0  13.0                     12.0  11.0     20.0  9.0     18.0  13.0     12.0  6.0       10.0  8.0
negative_wrong_or_missing_item        6.0   4.0                      4.0   1.0      0.0  1.0      5.0   2.0      1.0  0.0        1.0  1.0

In [6]:
print('% OF BAD ISSUES OVERALL')
issues=['missing_item', 'negative_food_packaging', 'negative_portion_size', 'negative_taste', 'negative_wrong_or_missing_item']
issues_shift_count = rests_issue_df.groupby(['Issue', 'Outlet name', 'Shift'])['Order number'].nunique().round(2).unstack(level=[1,2])
issues_shift_count = issues_shift_count.loc[issues,:].fillna(0)
issues_shift_count.loc['Total',:] = issues_shift_count.sum()
issues_shift_count.div(issues_shift_count.loc['Total', :], axis=1).drop('Total', axis='index').round(2)*100



% OF BAD ISSUES OVERALL


Outlet name                    Gachibowli       Istahgrill Himayat Nagar       Kondapur       Madhapur       Shaikpet        Yousufguda      
Shift                                   E     M                        E     M        E     M        E     M        E      M          E     M
Issue                                                                                                                                        
missing_item                          0.0   0.0                      5.0   0.0      0.0   0.0      0.0   5.0      0.0    0.0        6.0   0.0
negative_food_packaging               4.0  11.0                      5.0  13.0     13.0  29.0     13.0   9.0      9.0    0.0       12.0  28.0
negative_portion_size                17.0  29.0                     14.0  35.0     23.0  12.0     13.0  18.0     32.0    0.0       18.0  22.0
negative_taste                       54.0  46.0                     57.0  48.0     65.0  53.0     58.0  59.0     55.0  100.0       59.0  44.0
negative_wrong_or_missing_item       25.0  14.0                     19.0   4.0      0.0   6.0     16.0   9.0      5.0    0.0        6.0   6.0

In [7]:
print("ITEM WISE AVG RATING")
items_shift_rating = rests_issue_df.groupby(['Item', 'Outlet name', 'Shift'])['Rating'].mean().round(2).unstack(level=[1,2]).sort_index(axis='columns')
total_shift_ratings = rests_issue_df.groupby(['Item', 'Shift'])['Order number'].nunique().unstack(level=-1)
total_shift_ratings.columns = pd.MultiIndex.from_product([['Total'], total_shift_ratings.columns])
items_shift_rating = items_shift_rating.merge(total_shift_ratings, left_index=True, right_index=True)
items_shift_rating = items_shift_rating.sort_values(('Total', 'E'), ascending=False).fillna(0)
items_shift_rating.style.apply(lambda x: ['color: red' if 0 < v < 4 else ('color: blue' if 4 <= v < 4.3 else '') for v in x], axis=1)



ITEM WISE AVG RATING


In [13]:
print("ITEM WISE ISSUES")
item_issues = rests_issue_df.groupby(['Issue', 'Item'])['Order number'].nunique().round(2).unstack(level=0)
item_issues.loc[:, issues].sort_values('negative_wrong_or_missing_item', ascending=False)




ITEM WISE ISSUES


Issue,missing_item,negative_food_packaging,negative_portion_size,negative_taste,negative_wrong_or_missing_item
Item,,,,,
Spicy Chicken Shawarma,NaN,4.0,1.0,23.0,7.0
Istah's Chicken Shawarma (Rumali),NaN,4.0,6.0,22.0,5.0
Chicken Sahaan (Meat Bowl),1.0,4.0,6.0,17.0,4.0
Chicken Salad,NaN,7.0,13.0,17.0,2.0
Chicken Shawarma (Khubus),1.0,2.0,3.0,11.0,2.0
Chicken Shawarma (Rumali),NaN,2.0,4.0,23.0,2.0
Chicken Shawarma in Rumali + French Fries + Appy Fizz (250 Ml),NaN,NaN,1.0,1.0,2.0
Falafel Shawarma,NaN,NaN,1.0,3.0,2.0
Seethaphal by House of Pops,NaN,1.0,NaN,NaN,2.0


## Mark Food Ready Analysis

In [15]:
import requests
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display

cookie = 'visid_incap_1562703=ICse0eccTZKmIZE2eoF/jxX9dFwAAAAAQUIPAAAAAAC9+jEGXPuhaebtrEQJCe+U; nlbi_1562703=VZFNaiO+hASM9Ajyc0R+AwAAAABz84pLmQRj80CXM1WM1GAx; is_dot_in=1; 9677222952_prep_time_status=3; visid_incap_1554223=XD8B5PfJS1K73lqUAd2s16b9eFwAAAAAQUIPAAAAAABxqgMkpm3yn3qnwwhEqGqw; nlbi_1554223=I0+9M0UAa3fXw59Vo1DVjgAAAABbpdrsTm3eJ/ZuWQffruqv; _gcl_au=1.1.1519397198.1551433204; _ga=GA1.2.813094418.1551433205; 55969_prep_time_status=3; Swiggy_Session-newSys=1; incap_ses_747_1562703=2QqOer59VUyOsAd4j+BdCj0tiVwAAAAAZJSM/ZX9vPAoItLMXxd19g==; _gid=GA1.2.830474475.1553004822; incap_ses_1234_1554223=eWf8IK9exGllDmU2kgwgEWwMkVwAAAAAtwzRCrQqzBBNNkwi1ZJedw==; incap_ses_746_1562703=3u7qKN7BtFj2qBPKwVNaCqIykVwAAAAAhUm20S7dCmihICmcddsjWQ==; Swiggy_Session-alpha=f52c8068-8b62-497e-b153-18713ab3b818; Swiggy_Session-user=9677222952; Swiggy_Session-perms=%5B1%2C2%2C3%2C4%2C5%5D; Swiggy_user_role=1; Swiggy_change_password=false'
rest_ids = [29553,24766,27107,41976,47120,65483]

In [16]:
mfr_data = pd.read_csv('mark_food_ready.csv', parse_dates=['ordered_time'])
mfr_data['ordered_time'] = pd.to_datetime(mfr_data['ordered_time'])

@interact
def dateFilter(rest_id = rest_ids,
    start = mfr_data['ordered_time'].sort_values(ascending=False).apply(lambda x: x.strftime('%Y-%m-%d')).unique().tolist(),
    end = mfr_data['ordered_time'].sort_values(ascending=False).apply(lambda x: x.strftime('%Y-%m-%d')).unique().tolist()):
    mfr_shift =  mfr_data[(mfr_data['order_status'] == 'delivered') & (mfr_data['rest_id'] == rest_id) & (mfr_data['ordered_time'] >= start + ' 00:00:00') & (mfr_data['ordered_time'] <= end + ' 23:59:59')].groupby(['rest_id', 'shift', 'mfr_message'])['order_id'].nunique().unstack(level=-1).fillna(0)
    return mfr_shift.div(mfr_shift.sum(axis='columns'), axis='index').style.format('{:.2%}')



interactive(children=(Dropdown(description='rest_id', options=(29553, 24766, 27107, 41976, 47120, 65483), valu…

In [28]:
data = []

    
@interact_manual
def get_swiggy_mfr_data(start_date=widgets.DatePicker(description='Start date'), end_date = widgets.DatePicker(description='End date')):
    for rest_id in rest_ids:
        print(rest_id)
        more = ''
        offset = -20
        while more != None:
            offset += 20
            url = 'https://partner.swiggy.com/orders/v0/history/?limit=20&offset=' + str(offset) + '&ordered_time__gte=' + start_date.strftime('%Y-%m-%d') + '&ordered_time__lte=' + end_date.strftime('%Y-%m-%d') + '&restaurant_id=' + str(rest_id)
            # print(url)
            r = requests.get(url, headers={'Cookie': cookie})
            json_r = r.json()
            # print(json_r)
            more = json_r['data'][0]['data']['meta']['next']
            data_json = r.json()['data'][0]['data']['objects']
            for d in data_json:
                # print(d['order_id'])
                if len(d['mfrAccuracy']) != 0:
                    temp_js = {
                        'rest_id': rest_id,
                        'order_id': d['order_id'], 
                        'ordered_time': d['status']['ordered_time'], 
                        'order_status': d['status']['order_status'], 
                        'bill': d['bill'], 
                        'mfr_message': d['mfrAccuracy']['message'], 
                        'mfr_status': d['mfrAccuracy']['status']
                    }
                else:
                    temp_js = {
                        'rest_id': rest_id,
                        'order_id': d['order_id'], 
                        'ordered_time': d['status']['ordered_time'], 
                        'order_status': d['status']['order_status'], 
                        'bill': d['bill'], 
                        'mfr_message': None, 
                        'mfr_status': None
                    }
                data.append(temp_js)
    return "Done"



interactive(children=(DatePicker(value=None, description='Start date'), DatePicker(value=None, description='En…

In [4]:
data_df = pd.DataFrame(data)

data_df = data_df[['rest_id', 'order_id', 'order_status', 'ordered_time', 'mfr_message', 'mfr_status']]

In [5]:
data_df.groupby(['rest_id'])['order_id'].count()

rest_id
24766    272
27107    489
29553    602
41976    466
47120    578
65483    324
Name: order_id, dtype: int64

In [6]:
data_df['ordered_time'] = pd.to_datetime(data_df['ordered_time'])

In [7]:
data_df['shift'] = data_df['ordered_time'].apply(lambda x: 'M' if 8 <= x.hour <= 18 else 'E')
data_df.head()

,rest_id,order_id,order_status,ordered_time,mfr_message,mfr_status,shift
0,29553,36527309700,cancelled,2019-03-24 23:58:29,Food Ready Not pressed,0,E
1,29553,36527223142,delivered,2019-03-24 23:57:04,Food Ready Correct,1,E
2,29553,36526762713,delivered,2019-03-24 23:49:23,Food Ready Correct,1,E
3,29553,36526717368,delivered,2019-03-24 23:48:38,Food Ready Correct,1,E
4,29553,36526375763,delivered,2019-03-24 23:42:55,Food Ready Correct,1,E


In [24]:
mfr_data = pd.read_csv('mark_food_ready.csv', parse_dates=['ordered_time'])

mfr_data = pd.concat([mfr_data, data_df], axis='index', sort=False, ignore_index=True)
mfr_data = mfr_data.sort_values('ordered_time', ascending=False)


mfr_data.to_csv('mark_food_ready.csv', index=False)
mfr_data.head()

NameError: name 'data_df' is not defined